In [79]:
!pip install prophet

#### Import Libraries

In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
import os
from sklearn.metrics import mean_squared_log_error
import datetime
import math
from prophet import Prophet

#### Load data

In [81]:
dir_path = '/kaggle/input/store-sales-time-series-forecasting/'

holidays = pd.read_csv(dir_path + 'holidays_events.csv')
oil = pd.read_csv(dir_path + 'oil.csv')
stores = pd.read_csv(dir_path + 'stores.csv')
test = pd.read_csv(dir_path + 'test.csv')
train = pd.read_csv(dir_path + 'train.csv')
transactions = pd.read_csv(dir_path + 'transactions.csv')

#### Summary statistics

In [82]:
holidays.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [83]:
holidays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB


In [84]:
holidays.describe()

,date,type,locale,locale_name,description,transferred
count,350,350,350,350,350,350
unique,312,6,3,24,103,2
top,2014-06-25,Holiday,National,Ecuador,Carnaval,False
freq,4,221,174,174,10,338


In [85]:
holidays['type'].unique()

array(['Holiday', 'Transfer', 'Additional', 'Bridge', 'Work Day', 'Event'],
      dtype=object)

In [86]:
holidays['locale'].unique()

array(['Local', 'Regional', 'National'], dtype=object)

In [87]:
holidays['locale_name'].unique()

array(['Manta', 'Cotopaxi', 'Cuenca', 'Libertad', 'Riobamba', 'Puyo',
       'Guaranda', 'Imbabura', 'Latacunga', 'Machala', 'Santo Domingo',
       'El Carmen', 'Cayambe', 'Esmeraldas', 'Ecuador', 'Ambato',
       'Ibarra', 'Quevedo', 'Santo Domingo de los Tsachilas',
       'Santa Elena', 'Quito', 'Loja', 'Salinas', 'Guayaquil'],
      dtype=object)

In [88]:
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [89]:
oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1175 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


In [90]:
oil.describe()

,dcoilwtico
count,1175.000000
mean,67.714366
std,25.630476
min,26.190000
25%,46.405000
50%,53.190000
75%,95.660000
max,110.620000


In [91]:
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [92]:
oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1175 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


In [93]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [94]:
stores.describe()

,store_nbr,cluster
count,54.000000,54.000000
mean,27.500000,8.481481
std,15.732133,4.693395
min,1.000000,1.000000
25%,14.250000,4.000000
50%,27.500000,8.500000
75%,40.750000,13.000000
max,54.000000,17.000000


In [95]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [96]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB


In [97]:
train.isna().sum()

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

In [98]:
train.describe()

,id,store_nbr,sales,onpromotion
count,3.000888e+06,3.000888e+06,3.000888e+06,3.000888e+06
mean,1.500444e+06,2.750000e+01,3.577757e+02,2.602770e+00
std,8.662819e+05,1.558579e+01,1.101998e+03,1.221888e+01
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,7.502218e+05,1.400000e+01,0.000000e+00,0.000000e+00
50%,1.500444e+06,2.750000e+01,1.100000e+01,0.000000e+00
75%,2.250665e+06,4.100000e+01,1.958473e+02,0.000000e+00
max,3.000887e+06,5.400000e+01,1.247170e+05,7.410000e+02


In [99]:
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [100]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           28512 non-null  int64 
 1   date         28512 non-null  object
 2   store_nbr    28512 non-null  int64 
 3   family       28512 non-null  object
 4   onpromotion  28512 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.1+ MB


In [101]:
test.describe()

,id,store_nbr,onpromotion
count,2.851200e+04,28512.000000,28512.000000
mean,3.015144e+06,27.500000,6.965383
std,8.230850e+03,15.586057,20.683952
min,3.000888e+06,1.000000,0.000000
25%,3.008016e+06,14.000000,0.000000
50%,3.015144e+06,27.500000,0.000000
75%,3.022271e+06,41.000000,6.000000
max,3.029399e+06,54.000000,646.000000


In [102]:
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [103]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          83488 non-null  object
 1   store_nbr     83488 non-null  int64 
 2   transactions  83488 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.9+ MB


#### Data Pre-proccessing

In [104]:
# transform sales to log scale
train['sales'] = np.log1p(train['sales'])

In [105]:
# convert date field into a date
oil['date'] = pd.to_datetime(oil['date'], format = "%Y-%m-%d")
holidays['date'] = pd.to_datetime(holidays['date'], format = "%Y-%m-%d")
transactions['date'] = pd.to_datetime(transactions['date'], format = "%Y-%m-%d")
train['date'] = pd.to_datetime(train['date'], format = "%Y-%m-%d")
test['date'] = pd.to_datetime(test['date'], format = "%Y-%m-%d")

In [106]:
# fill missing values
oil['dcoilwtico'] = oil['dcoilwtico'].fillna(method='bfill')

In [107]:
# from fbprophet import Prophet

# Dictionary to store store-level forecasts
store_level_forecasts = {}

# Iterate through each store
for store in range(1, 55):
    store_data = train[train['store_nbr'] == store]
    
    # Prepare data for Prophet at store level
    store_data_prophet = store_data.rename(columns={'date': 'ds', 'sales': 'y'})
    
    # Initialize and fit the Prophet model for the store
    model = Prophet()
    model.fit(store_data_prophet)
    
    # Make future dates for forecasting
    future = model.make_future_dataframe(periods=len(test))
    
    # Forecast sales for the store
    forecast = model.predict(future)
    store_level_forecasts[store] = forecast


16:49:05 - cmdstanpy - INFO - Chain [1] start processing
16:49:23 - cmdstanpy - INFO - Chain [1] done processing
16:49:39 - cmdstanpy - INFO - Chain [1] start processing
16:50:05 - cmdstanpy - INFO - Chain [1] done processing
16:50:21 - cmdstanpy - INFO - Chain [1] start processing
16:50:47 - cmdstanpy - INFO - Chain [1] done processing
16:51:04 - cmdstanpy - INFO - Chain [1] start processing
16:51:16 - cmdstanpy - INFO - Chain [1] done processing
16:51:32 - cmdstanpy - INFO - Chain [1] start processing
16:51:46 - cmdstanpy - INFO - Chain [1] done processing
16:52:03 - cmdstanpy - INFO - Chain [1] start processing
16:52:11 - cmdstanpy - INFO - Chain [1] done processing
16:52:27 - cmdstanpy - INFO - Chain [1] start processing
16:52:39 - cmdstanpy - INFO - Chain [1] done processing
16:52:55 - cmdstanpy - INFO - Chain [1] start processing
16:53:16 - cmdstanpy - INFO - Chain [1] done processing
16:53:32 - cmdstanpy - INFO - Chain [1] start processing
16:53:49 - cmdstanpy - INFO - Chain [1]

In [108]:
# from fbprophet import Prophet

# Dictionary to store forecasted family proportions for each store
forecasted_family_proportions = {}

# Iterate through each store
for store in range(1, 55):
    store_data = train[train['store_nbr'] == store]
    
    # Perform calculations to get family proportions for the store (assuming 'family' column exists)
    family_proportions = store_data.groupby('family')['sales'].sum() / store_data['sales'].sum()
    
    # Create a mapping of numerical identifiers for families
    family_mapping = {family: idx for idx, family in enumerate(family_proportions.index)}
    
    # Map family names to numerical IDs in the original data
    store_data['family_numeric'] = store_data['family'].map(family_mapping)
    
    # Prepare data for Prophet for forecasting family proportions using numeric identifiers
    family_proportions_df = store_data.groupby('date')['family_numeric'].mean().reset_index()
    family_proportions_df.columns = ['ds', 'y']
    
    # Initialize and fit the Prophet model for forecasting family proportions
    model = Prophet()
    model.fit(family_proportions_df)
    
    # Make future dates for forecasting family proportions
    future = pd.DataFrame({'ds': pd.date_range(start=train['date'].min(), end=test['date'].max())})
    
    # Forecast family proportions for the store
    forecast = model.predict(future)
    forecasted_family_proportions[store] = forecast


In [109]:
# Dictionary to store store-family level forecasts
store_family_forecasts = {}

# Iterate through each store
for store in range(1, 55):
    # Get the forecast for the store from Step 1
    store_forecast = store_level_forecasts[store]
    
    # Get the forecasted family proportions for the store from Step 2
    forecasted_proportions = forecasted_family_proportions[store]
    
    # Merge the store forecast and forecasted family proportions based on the date 'ds'
    merged_forecast = pd.merge(store_forecast, forecasted_proportions, on='ds', how='inner')
    
    # Calculate store-family level forecast by multiplying store-level forecast by family proportion
    merged_forecast['store_family_forecast'] = merged_forecast['yhat_x'] * merged_forecast['yhat_y']
    
    # Store the store-family level forecast
    store_family_forecasts[store] = merged_forecast[['ds', 'store_family_forecast']]


In [110]:
# from fbprophet import Prophet
# import pandas as pd
import csv

# Assuming 'train', 'test', 'store_level_forecasts', 'forecasted_family_proportions' are available

# ... (Previous code for forecasting store and family proportions)

# Dictionary to store store-family level forecasts
store_family_forecasts = {}

# Iterate through each store
for store in range(1, 55):
    # Get the forecast for the store from Step 1
    store_forecast = store_level_forecasts[store]
    
    # Get the forecasted family proportions for the store from Step 2
    forecasted_proportions = forecasted_family_proportions[store]
    
    # Merge the store forecast and forecasted family proportions based on the date 'ds'
    merged_forecast = pd.merge(store_forecast, forecasted_proportions, on='ds', how='inner')
    
    # Calculate store-family level forecast by multiplying store-level forecast by family proportion
    merged_forecast['store_family_forecast'] = merged_forecast['yhat_x'] * merged_forecast['yhat_y']
    
    # Store the store-family level forecast
    store_family_forecasts[store] = merged_forecast[['ds', 'store_family_forecast']]

# Dictionary to store predicted sales for each ID in the test set
predicted_sales = {}

# Iterate through test data to extract predicted sales
for index, row in test.iterrows():
    store_nbr = row['store_nbr']
    date = row['date']
    
    # Fetch the store-family level forecast for the respective store and date
    store_family_forecast = store_family_forecasts[store_nbr]
    forecast_row = store_family_forecast[store_family_forecast['ds'] == date]
    
    # Extract the predicted sales value for the current ID (assuming 'id' is the column name in 'test')
    predicted_sales[row['id']] = forecast_row['store_family_forecast'].values[0]

# File path for saving the submission file
submission_file = 'submission.csv'

# Write predicted sales values to the submission file
with open(submission_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['id', 'sales'])  # Header
    for id_value, sales_value in predicted_sales.items():
        writer.writerow([id_value, sales_value])
